# Лабораторная работа №1
## ----------------------------------------------------------------------------------------------------------------------------------
## Задание:
1 . Провести классификацию найденного датасета, методами наивного Байеса  . В формате Markdown написать пояснения. Объяснить почему были выбраны именно такие гиперпараметры, была ли перекрестная проверка, и т.д.

## Решение:
### Ищем лучшие метрики с использованием 


In [5]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
import matplotlib.pyplot as plt
from imblearn.under_sampling import RandomUnderSampler

Dataset = pd.read_csv('Dataset.csv')  

label_encoder = LabelEncoder()
Dataset['hiring_event_job'] = label_encoder.fit_transform(Dataset['company'])

non_numeric_columns = Dataset.select_dtypes(exclude=['number']).columns.tolist()
Dataset = Dataset.drop(columns=non_numeric_columns)

columns_to_fill = ['source_id', 'ad_id', 'hires_needed_exact', 'review_count']
for column in columns_to_fill:
    mean_value = Dataset[column].mean()
    Dataset[column].fillna(mean_value, inplace=True)

X = Dataset.drop('hiring_event_job', axis=1) 
y = Dataset['hiring_event_job']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

under_sampler = RandomUnderSampler(random_state=42)


X_resampled, y_resampled = under_sampler.fit_resample(X_train, y_train)

naive_bayes_models = {
    'GaussianNB': GaussianNB(),
    'MultinomialNB': MultinomialNB(),
    'BernoulliNB': BernoulliNB(),
}

param_grid = {
    'GaussianNB': {},
    'MultinomialNB': {'alpha': [0.1, 0.5, 1.0]},
    'BernoulliNB': {'alpha': [0.1, 0.5, 1.0], 'binarize': [0.0, 0.1, 0.2]},
}

scoring_metric = 'accuracy'

best_models = {}
for model_name, model in naive_bayes_models.items():
    grid_search = GridSearchCV(model, param_grid[model_name], scoring=scoring_metric, cv=5)
    grid_search.fit(X_resampled, y_resampled)
    
    best_models[model_name] = grid_search.best_estimator_

best_model_name = max(best_models, key=lambda k: grid_search.cv_results_['mean_test_score'][grid_search.best_index_])
best_model = best_models[best_model_name]

print(f"Лучшая модель: {best_model_name}")
print(f"Лучшие параметры: {grid_search.best_params_}")


Лучшая модель: GaussianNB
Лучшие параметры: {'alpha': 0.1, 'binarize': 0.0}


### обучаем на лучьших гиперпараметрах

In [7]:
best_model.fit(X_resampled, y_resampled)

y_pred_full = best_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred_full)
classification_rep = classification_report(y_test, y_pred_full)
print(f'Точность модели: {accuracy}')
print(classification_rep)

Точность модели: 0.3333125
              precision    recall  f1-score   support

           0       0.33      0.27      0.30      5306
           1       0.34      0.39      0.36      5416
           2       0.33      0.33      0.33      5278

    accuracy                           0.33     16000
   macro avg       0.33      0.33      0.33     16000
weighted avg       0.33      0.33      0.33     16000

